In [38]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import datetime
import os.path

df = pd.DataFrame()
strippedrows = []

def getData(res):
    for row in soup.findAll("a", class_="search_result_row"):
        win = 0
        mac = 0
        lnx = 0

        #title
        #find <span> elements with the title class
        titlespan = row.find("span", class_="title").text

        #rating 
        #get only the rating value from the tooltip data
        try:
            rating = row.find("span", class_="search_review_summary")
            rating = rating['data-tooltip-html'].split('<br>')[1]
            rating = rating.split('%')
            rating = rating[0]+'/100'
        except:
            rating = "No rating(E)"
        
        #numreviews
        #get the amount of reviews from the tooltip data
        try:
            numreviews = row.find("span", class_="search_review_summary")
            numreviews = numreviews['data-tooltip-html'].split('<br>')[1]
            numreviews = numreviews.replace(",","")
            numreviews = re.findall(r'\d+', numreviews)[1]
        except:
            numreviews = "No reviews(E)"

        #discount%
        #check if there is a discount
        try:
            rabatt = row.find("div", class_="discount_pct").text
            
            if(bool(re.search(r'\d',rabatt))):
                rabatt = rabatt.replace('\n','')
            else:
                rabatt = "Unknown"
        except:
            rabatt = "Unknown(E)"
            
        #price-------
        #check for a price
        try:
            price = row.find("div", class_="discount_final_price").text

            if(bool(re.search(r'\d', price))):
                price = price.replace('\n','')
                price = price.split("€")
                price = price[0]
                
            else:
                price = "Unknown"
        except:
            price = "Unknown(E)"

        #regprice-------
        #check if the previous price exists
        try:
            regprice = row.find("div", class_="discount_original_price").text
            
            if(bool(re.search(r'\d', regprice))):
                regprice = regprice.replace('\n','')
                regprice = regprice.split("€")
                regprice = regprice[0]
            else:
                regprice = "Unknown"
        except:
            regprice = "Unknown(E)"

        #dateyear
        #yoinks the releaseyear if exists
        dateyear = row.find("div", class_="search_released").text

        if(bool(re.search(r'\d', dateyear))):
            try:
                dateyear = dateyear.split(",")
                dateyear = dateyear[1].replace(' ','')
            except:
                dateyear = "Unknown"
        else:
            dateyear = "Unknown(E)"

        #platform
        #check which platforms the game supports: windows, linux & mac
        platform = row.findAll("span", class_="platform_img")
        for classes in platform:
            postclass = classes.attrs['class'][1]
            if(postclass == 'win'):
                win = 1
            elif(postclass == 'mac'):
                mac = 1
            elif(postclass == 'linux'):
                lnx = 1
        #append all variables to a list
        strippedrows.append([titlespan, rating, numreviews, rabatt, price, regprice, dateyear, win, mac, lnx, time.strftime("%d-%m-%Y-%H:%M:%S")])

#send requests to get a certain amount of pages from the steam specials website (around 25 items per page)
for pagenr in range (1,5): 
    res = requests.get("https://store.steampowered.com/search/?specials=1&page=" + str(pagenr))
    page = res.text
    soup = BeautifulSoup(page, "html.parser")
    time = datetime.datetime.now()
    getData(res)
    
#create the dataframe and set column names
df = pd.DataFrame(strippedrows)
df.columns = ['title','rating','#reviews','discount','price','regprice','releaseyear','win','mac','lnx','time']

#pd.options.display.max_rows = None
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [39]:
df

,title,rating,#reviews,discount,price,regprice,releaseyear,win,mac,lnx,time
0,EA SPORTS FC™ 25,47/100,41493,-60%,"27,99","69,99",2024,1,0,0,21-01-2025-23:29:08
1,Hearts of Iron IV,91/100,233045,-70%,"14,99","49,99",2024,1,1,1,21-01-2025-23:29:08
2,Stellaris,88/100,124197,-75%,"9,99","39,99",2016,1,1,1,21-01-2025-23:29:08
3,Crusader Kings III,91/100,77627,-70%,"14,99","49,99",2020,1,1,1,21-01-2025-23:29:08
4,Manor Lords,87/100,55944,-30%,"27,99","39,99",2024,1,0,0,21-01-2025-23:29:08
5,Forza Horizon 5,88/100,192234,-50%,"29,99","59,99",2021,1,0,0,21-01-2025-23:29:08
6,Stellaris: Ultimate Bundle,84/100,140712,-46%,"161,28","300,56",Unknown(E),1,1,1,21-01-2025-23:29:08
7,FINAL FANTASY VII REBIRTH,No rating(E),No reviews(E),-30%,"48,99","69,99",2025,1,0,0,21-01-2025-23:29:08
8,Tom Clancy's Rainbow Six® Siege,84/100,1149156,-60%,"7,99","19,99",2015,1,0,0,21-01-2025-23:29:08
9,Europa Universalis IV,87/100,89237,-90%,"4,99","49,99",2013,1,1,1,21-01-2025-23:29:08
